## Tecnológico de Costa Rica

### Autor: Jorge Andrés Brenes Alfaro

## Red mimetizadora



## 1. Bibliotecas

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dense, Dropout, GRU, TimeDistributed

import warnings
warnings.filterwarnings('ignore')

## 2. Dataset 

In [ ]:
Dir = os.listdir('/Users/jorge/Documents/TEC/TFG/Datos_Recolectados')
Data_Collect = np.array([[0,0,0,0]])
for filename in Dir:
    file = pd.read_csv('/Users/jorge/Documents/TEC/TFG/Datos_Recolectados/'+filename)
    Data_Collect = np.append(Data_Collect, file.values,axis=0)
train_data = Data_Collect[:,2]
train_label = Data_Collect[:,3]


train_data = np.array([])
val_data = np.array([])
train_label = np.array([])
val_label = np.array([])
cont=0

while(cont < Data_Collect[:,2].shape[0]):
    if cont < round(Data_Collect[:,2].shape[0]*0.75):
        train_data = np.append(train_data, Data_Collect[:,2][cont])
        train_label = np.append(train_label, Data_Collect[:,3][cont])
    else:
        val_data = np.append(val_data, Data_Collect[:,3][cont])
        val_label = np.append(val_label, Data_Collect[:,3][cont])
    cont+=1

train_data = np.reshape(train_data,(1,train_data.shape[0],1))
train_label = np.reshape(train_label,(1,train_label.shape[0],1))

val_data = np.reshape(val_data,(1,val_data.shape[0],1))
val_label = np.reshape(val_data,(1,val_label.shape[0],1))

In [ ]:
train_data.shape

## 3. Neural Network

### 3.1 Model Creation

In [ ]:
clear_session()

model = Sequential()
model.add(GRU(64, input_shape=(train_data.shape[1],1),return_sequences=True))
model.add(Dropout(0.35))
model.add(TimeDistributed(Dense(1)))  #there is no difference between this and model.add(Dense(1))...
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mse','acc'])

model.summary()

### 3.2 Model Training

In [ ]:
history = model.fit(train_data, train_label,
                    epochs=500, batch_size=8,
                    validation_data = (val_data,val_label),
                    verbose=2)

#testPredict = model.predict(test_data)

### 3.3 Model Evaluate

In [ ]:
#loss, accuracy = model.evaluate(testX,output_test)

In [ ]:
joblib.dump(model, 'GRU_model.joblib')

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.figure()
plt.plot(epochs, loss,'b', label='Training loss')
plt.plot(epochs, val_loss,'r', label='Validation loss')
plt.title('Training and validation losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xscale('log')
plt.legend()
plt.show()